In [67]:
# Imports
from collections import defaultdict
import csv
import dateutil.parser
import HTMLParser
from nearest_words import read_file, word_to_embedding
import nltk
from nltk.tokenize import casual_tokenize
import numpy as np
import os

In [84]:
# Constants
PATH = '../data/'
EMBEDDINGS_FILE = 'glove.twitter.27B.50d.txt'
MISSING_WORDS_FILE = 'missing_words.txt'
WORDS_FILE = 'words.txt'
TWEET_CNTS_FILE = 'tweet_counts.txt'
MIN_DIST = 4.5
NUM_MONTHS = 35
MIN_TWEETS = 100 # minimum number of tweets required per city-month
K = 50 # number of tweets per batch
NUM_RESAMPLES = 8 # number of times to resample from city-month tweets to generate batches

In [10]:
# Global variables
word_to_idx = dict() # word to index in glove embeddings file
city_to_changes = defaultdict(lambda: defaultdict(lambda: [0, 0])) # city to month to [trend, spike]

In [54]:
def create_words_file():
    print 'Begin creating words file...'
    with open(WORDS_FILE, 'w') as wf:
        with open(PATH + EMBEDDINGS_FILE, 'r') as ef:
            idx = 0
            for line in ef:
                word = line.split()[0]
                wf.write(word + '\n')
                word_to_idx[word] = idx
                idx += 1
    print 'Finished creating words file!'

In [55]:
create_words_file()

Begin creating words file...
Finished creating words file!


In [ ]:
def output_tweet_counts(month_to_tweets):
    with open(TWEET_CNTS_FILE, 'w') as f:
        f.write(','.join([str(len(month_to_tweets[month])) for month in month_to_tweets]))

In [ ]:
def output_missing_words(missing_words):
    with open(MISSING_WORDS_FILE, 'w') as f:
        for word in missing_words:
            f.write(word + '\n')

In [65]:
def output_embeddings(month_to_tweets, city):
    print 'Begin outputting word embeddings...'
    for month in month_to_tweets:
        if (len(month_to_tweets[month]) >= MIN_TWEETS):
            with open('%s/embeddings/%s_%d_embeddings.csv' % (PATH[:-1], city, month), 'w') as output:
                for tweet in month_to_tweets[month]:
                    output.write(','.join([str(num) for num in tweet]) + '\n')
    print 'Finished outputting word embeddings!'

In [74]:
def get_word_embeddings():
    print 'Begin getting word embeddings...'
    tweets_file = 'Delhi_tweets.csv'
    title = 'delhi'
    cnt = 0
    with open(PATH + tweets_file) as csvfile:
        month_to_tweets = defaultdict(list)
        reader = csv.DictReader(csvfile)
        missing_words = set()
        missing_words_cnt = 0
        for row in reader:
            if (cnt % 100000 == 0): print str(cnt) + ' tweets processed...'
            cnt += 1
            date = dateutil.parser.parse(row['postedTime'])
            month_idx = (date.year - 2014)*12 + (date.month - 1)
            tweet = ' '.join([word for word in casual_tokenize(row['tweet']) 
                              if '@' not in word and 'http' not in word and '#' not in word])
            tweet = tweet.lower()
            tweet_embedding = []
            for word in tweet.split():
                if (word in word_to_idx):
                    tweet_embedding.append(word_to_idx[word])
                else:
                    missing_words_cnt += 1
                    missing_words.add(word)
            if (len(tweet_embedding) > 0): month_to_tweets[month_idx].append(tweet_embedding)
            output_embeddings(month_to_tweets, title)
            output_tweet_counts(month_to_tweets)
            output_missing_words(missing_words)
    print 'Finished getting word embeddings!'
    print 'Number of missing words: ' + str(missing_words_cnt)

In [1]:
get_word_embeddings()

NameError: name 'get_word_embeddings' is not defined

In [2]:
def get_deviations():
    print 'Getting deviations...'
    with open(PATH + 'India_Onion_Prices_Vector.csv') as csvfile:
        reader = csv.reader(csvfile)
        next(reader, None)
        for row in reader:
            city = row[0]
            trends = row[2:NUM_MONTHS + 2]
            spikes = row[NUM_MONTHS + 3:2*NUM_MONTHS + 3]
            for i in range(NUM_MONTHS):
                trend = int(trends[i]) + 1 if (trends[i] != 'NA') else -1
                spike = int(spikes[i]) if (spikes[i] != 'NA') else -1
                city_to_changes[city][i] = [trend, spike]
    print ' -done.'

In [48]:
def is_relevant(tweet, keyword_embedding):
    for word in tweet.split():
        if (word in word_to_embedding):
            embedding = np.array(word_to_embedding[word])
            dist = np.linalg.norm(embedding - keyword_embedding)
            if (dist <= MIN_DIST):
                return True
    return False

In [75]:
def output_batches(month_to_tweets, city):
    num_batches = 0
    for month in month_to_tweets:
        tweet_count = len(month_to_tweets[month])
        trend = city_to_changes[city][month][0]
        spike = city_to_changes[city][month][1]
        if (tweet_count >= MIN_TWEETS and trend != -1 and spike != -1):
            num_batches += create_batches(tweet_count, city, month, trend, spike)
    return num_batches

In [91]:
def create_batches(tweet_count, city_name, month_idx, trend, spike):
    np.random.seed(10)
    
    n = tweet_count
    rand_seq = []
    for i in range(NUM_RESAMPLES):
        cur = [str(num) for num in np.random.choice(n, n, replace=False)]
        remaining = K - (n % K) # used to make the length of rand_seq a multiple of K
        cur += [str(num) for num in np.random.choice(n, remaining, replace=False)]
        rand_seq += cur

    num_batches = len(rand_seq) / K
    folder = 'batches'
    output_file = '%s%s/%s_%s_batch.txt' % (PATH, folder, city_name, str(month_idx))
    with open(output_file, 'w') as output:
        output.write('%d,%d\n' % (trend, spike))
        for i in range(num_batches):
            suffix = '\n'
            if (i == num_batches - 1): suffix = ''
            output.write('\t'.join(rand_seq[i * K : (i + 1) * K]) + suffix)
    
    with open(PATH + 'batch_counts.txt', 'a') as f:
        f.write('%s,%d,%d\n' % (city_name, month_idx, num_batches))
        
    return num_batches

In [63]:
def get_relevant_word_embeddings(keyword):
    keyword_embedding = np.array(word_to_embedding[keyword])
    total_batches = 0
    
    for tweets_file in os.listdir(PATH):
        if ('.csv' not in tweets_file) or ('India_Onion_Prices' in tweets_file): continue
        if ('Mumbai' not in tweets_file): continue
        print 'Reading ' + tweets_file + '...'
        city = tweets_file.split('_')[0]
        tweets_processed = 0
        month_to_tweets = defaultdict(list)

        with open(PATH + tweets_file) as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                if (tweets_processed % 1e5 == 0): print str(tweets_processed) + ' tweets processed...'
                tweets_processed += 1
                date = dateutil.parser.parse(row['postedTime'])
                month_idx = (date.year - 2014)*12 + (date.month - 1)
                tweet = ' '.join([word for word in casual_tokenize(row['tweet']) 
                                  if '@' not in word and 'http' not in word and '#' not in word])
                tweet = tweet.lower()
                tweet_embedding = []
                if (is_relevant(tweet, keyword_embedding)):
                    for word in tweet.split():
                        if (word in word_to_idx):
                            tweet_embedding.append(word_to_idx[word])
                if (len(tweet_embedding) > 0): 
                    month_to_tweets[month_idx].append(tweet_embedding)
                
        output_embeddings(month_to_tweets, city)
        
#         print 'Outputting batches...'
#         batches_created = output_batches(month_to_tweets, city)
        
#         total_batches += batches_created
#         print 'Batches so far: %d' % total_batches
#         print ' -done.'
    
#     print 'Total batches: %d' % total_batches

In [11]:
# Reads embeddings files and creates word_to_embedding map
read_file(PATH + EMBEDDINGS_FILE)

Reading file...
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
 -done


In [12]:
# Stores all price changes and spikes in city_to_changes
get_deviations()

Getting deviations...
 -done.


In [64]:
# Creates all tweet embeddings related to given parameter and outputs batches
get_relevant_word_embeddings('onion')

Reading Mumbai_tweets.csv...
0 tweets processed...
100000 tweets processed...
200000 tweets processed...
300000 tweets processed...
400000 tweets processed...
500000 tweets processed...
600000 tweets processed...
700000 tweets processed...
800000 tweets processed...
900000 tweets processed...
1000000 tweets processed...
1100000 tweets processed...
1200000 tweets processed...
1300000 tweets processed...
1400000 tweets processed...
1500000 tweets processed...
1600000 tweets processed...
1700000 tweets processed...
1800000 tweets processed...
1900000 tweets processed...
2000000 tweets processed...
2100000 tweets processed...
2200000 tweets processed...
2300000 tweets processed...
2400000 tweets processed...
2500000 tweets processed...
2600000 tweets processed...
2700000 tweets processed...
2800000 tweets processed...
2900000 tweets processed...
3000000 tweets processed...
3100000 tweets processed...
3200000 tweets processed...
3300000 tweets processed...
3400000 tweets processed...
350000

In [96]:
city_to_tweets = defaultdict(lambda: [0 for i in range(NUM_MONTHS)])

# Read embeddings files
def read_embeddings_files():
    path = '../data/embeddings/'
    total = 0
    for embeddings_file in os.listdir(path):
        filename = embeddings_file.split('_')
        city = filename[0]
        month = int(filename[1])
        with open(path + embeddings_file, 'r') as csvfile:
            num_tweets = sum(1 for row in csv.reader(csvfile))
            city_to_tweets[city][month] += num_tweets
            total += num_tweets
    print 'Total number of tweets: %d' % total
            
# Outputs batches by reading embeddings files to get tweet counts
def output_batches_1():
    num_batches = 0
    for city in city_to_tweets:
        city_batches = 0
        for month in range(NUM_MONTHS):
            tweet_count = city_to_tweets[city][month]
            trend = city_to_changes[city][month][0]
            spike = city_to_changes[city][month][1]
            if (tweet_count >= MIN_TWEETS and trend != -1 and spike != -1):
                batches = create_batches(tweet_count, city, month, trend, spike)
                city_batches += batches
        
        num_batches += city_batches
        print 'city: %s, batches: %d' % (city, city_batches)
        
    print 'Total number of batches: %d' % num_batches

read_embeddings_files()
output_batches_1()

Total number of tweets: 638606
city: Chandigarh, batches: 1328
city: Gurgaon, batches: 4040
city: Chennai, batches: 1992
city: Lucknow, batches: 1376
city: Hyderabad, batches: 4128
city: Delhi, batches: 24560
city: Mumbai, batches: 38520
city: Bengaluru, batches: 18424
city: Jaipur, batches: 1264
city: Kolkata, batches: 6952
Total number of batches: 102584
